# Q1. Running Elastic

In [ ]:
!pip install tiktoken

In [71]:
from elasticsearch import Elasticsearch
import json
from tqdm.auto import tqdm
import tiktoken
from openai import OpenAI

In [6]:
es_client = Elasticsearch('http://localhost:9200') 

In [7]:
es_client.info().body['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

# Q2. Indexing the data

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-vrn"


In [ ]:
es_client.indices.create(index=index_name, body=index_settings)

In [10]:
with open('documents.json', 'rt') as file_in:
    docs_raw = json.load(file_in)

In [12]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████| 948/948 [00:23<00:00, 39.93it/s]


# Q3. Searching

In [46]:
query = 'How do I execute a command in a running docker container?'

In [47]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
#            },
#            "filter": {
#                "term": {
#                    "course": "data-engineering-zoomcamp"
#                }
            }
        }
    }
}
    
response = es_client.search(index=index_name, body=search_query)

In [48]:
response['hits']['hits'][0]['_score']

84.050095

# Q4. Filtering

In [49]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [55]:
response = es_client.search(index=index_name, body=search_query)

In [56]:
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [51]:
response['hits']['hits'][2]['_source']['question']
#response

'How do I copy files from a different folder into docker container’s working directory?'

# Q5. Building a prompt

In [61]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip() 
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"{context_template.format(question=doc['question'], text=doc['text'])}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [62]:
prompt = build_prompt(query, result_docs)

In [64]:
len(prompt)

1462

# Q6. Tokens

In [68]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [69]:
len(encoding.encode(prompt))

322

# Extra's

In [72]:
client = OpenAI()

In [73]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'To execute a command in a running Docker container, you should first find the container ID by running the `docker ps` command. Once you have the container ID, use the `docker exec` command to execute the desired command inside the container. \n\nHere is how you can do it step-by-step:\n\n1. Find the container ID:\n   ```sh\n   docker ps\n   ```\n\n2. Execute a command inside the container (e.g., start a bash session):\n   ```sh\n   docker exec -it <container-id> bash\n   ```'

In [75]:
response.usage.prompt_tokens

442

In [76]:
response.usage.completion_tokens

113

In [77]:
response.usage.total_tokens

442